# Loading neurotransmitter and creating connectivity subgraphs for specific neurotransmitters
### Here we build upon the previous script to take putative neurotransmitters for each connection between neurons and make subgraphs that contain one or two neurotransmitters. 

In [ ]:
from cedne import utils

In [ ]:
w = utils.makeWorm()
nn = w.networks["Neutral"]
utils.loadNeurotransmitters(nn)

### Fold the network by neuron types (DB, VA, VC, etc.). This automatically copies the data from each of the neurons in the class to the neuron that represents the class. This aids in comaparing different sources of datasets with different formats with each other.

In [ ]:
nn_class = utils.foldByNeuronType(nn)

In [ ]:
nn_class.neurons['CAN'].type = 'interneuron' ## For the purposes of plotting

In [ ]:
conns = []
nt_rec = ('Glutamate', 'glc-2')
connFilter = 'chemical-synapse' #'gap-junction'
for c,e in nn_class.connections.items():
    if connFilter:
        if e.edge_type == connFilter:
            #print(e.putative_neurotrasmitter_receptors)
            if nt_rec in e.putative_neurotrasmitter_receptors:
                conns.append([c[0].name, c[1].name])

In [ ]:
pos = utils.plot_layered(conns, nn_class, nodeColors={}, edgeColors = 'gray', save=False, title=nt_rec, extraNodes=[], extraEdges=[], pos=[], mark_anatomical=False, colorbar=False)

### Plotting all glutamatergic connections with differnet receptor### Plotting all putative glutamatergic connections with differnet receptors.

In [ ]:
conns = {}
ligands= ('Glutamate')
connFilter = 'chemical-synapse' #'gap-junction'
for c,e in nn_class.connections.items():
    if connFilter:
        if e.edge_type == connFilter:
            #print(e.putative_neurotrasmitter_receptors)
            if ligands in e.ligands:
                for nt_rec in e.putative_neurotrasmitter_receptors:
                    if nt_rec[0] in ligands:
                        if not nt_rec in conns:
                            conns[nt_rec] = []
                        conns[nt_rec].append([c[0].name, c[1].name])

In [ ]:
for ntr, conn in conns.items():
    print(ntr)
    pos = utils.plot_layered(conn, nn_class, nodeColors={}, edgeColors = 'gray', save=False, title=ntr, extraNodes=[], extraEdges=[], pos=[], mark_anatomical=False, colorbar=False)